Test to chunk and make embeddings

In [3]:
import pandas as pd

In [3]:
##searching if their is missing data in the csv
##missing 31 stories because riot still doesn't wrote it 
df = pd.read_csv("../data/raw/lore.csv")
print(df.isnull().sum())
null_data = df[df.isnull().any(axis=1)]
df.fillna("L'éditeur Riot Games n'a pas encore fourni l'histoire de ce champion", inplace = True)
df
#null_data

Name          0
Region        0
SumUp         0
Biography     0
Story        31
dtype: int64


,Name,Region,SumUp,Biography,Story
0,AATROX,RUNETERRA,"Autrefois, Aatrox et ses frères étaient honoré...","Qu'on la prenne pour un dieu ou pour un démon,...",Les ténèbres. Le souffle que je ne peux pas pr...
1,AHRI,IONIA,"Connectée à la magie du royaume spirituel, Ahr...","Pendant le plus clair de sa vie, Ahri n'a rien...",Le marché embaumait l'encens et les légumes po...
2,AKALI,IONIA,Ayant abandonné l'Ordre Kinkou et le titre de ...,Ionia a toujours été une terre de magie sauvag...,« Aïe… Hé ! Bo'lii ! Tu n'y vas pas de main mo...
3,AKSHAN,SHURIMA,"Se jouant du danger, Akshan combat le mal sans...","Agissant dans l'ombre, un preux justicier traq...",Shadya n'était morte que depuis quelques semai...
4,ALISTAR,RUNETERRA,Alistar est un guerrier redoutable cherchant à...,"Bien des civilisations ont résisté à Noxus, ma...",L'éditeur Riot Games n'a pas encore fourni l'h...
...,...,...,...,...,...
166,ZERI,ZAUN,Zeri est une jeune femme vive et téméraire ori...,"Originaire d'une famille ouvrière, Zeri a gran...","« Je ne peux pas accepter », déclara le commer..."
167,ZIGGS,ZAUN,Amoureux des grosses bombes et des mèches cour...,Ziggs naquit avec un talent certain pour le br...,"Très cher Zaun. Je suis là, je suis pelucheux ..."
168,ZILEAN,RUNETERRA,"Autrefois membre du conseil d'Icathia, Zilean ...",Icathia n'a pas toujours été la plus désolée d...,L'éditeur Riot Games n'a pas encore fourni l'h...
169,ZOÉ,TARGON,"Incarnation de l'espièglerie, de l'imagination...",Conformément à la nature de la Manifestation t...,"Dès qu'elle pensa à la boutique de gâteaux, Zo..."


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)

liste_chunk = []

for index, row in df.iterrows():
    text_biography = row['Biography']
    text_story = row['Story']
    texts_bio = text_splitter.create_documents([text_biography],[{"Source": "Biography", "Name": row['Name'], "Region": row['Region'], "Sum_up": row['SumUp']}])
    text_story = text_splitter.create_documents([text_story], [{"Source": "Story", "Name": row['Name'], "Region": row['Region'], "Sum_up": row['SumUp']}])
    liste_chunk.extend(texts_bio)
    liste_chunk.extend(text_story)


liste_chunk[1].__dict__


{'id': None,
 'metadata': {'Source': 'Biography',
  'Name': 'AATROX',
  'Region': 'RUNETERRA',
  'Sum_up': "Autrefois, Aatrox et ses frères étaient honorés pour avoir défendu Shurima contre le Néant. Mais ils finirent par devenir une menace plus grande encore pour Runeterra : la ruse et la sorcellerie furent employées pour les battre. Cependant, après des siècles d'emprisonnement, Aatrox fut le premier à retrouver sa liberté, en corrompant et transformant les mortels assez stupides pour tenter de s'emparer de l'arme magique qui contenait son essence. Désormais en possession d'un corps qu'il a approximativement transformé pour rappeler son ancienne forme, il arpente Runeterra en cherchant à assouvir sa vengeance apocalyptique."},
 'page_content': "Shurima fut conduit devant le Disque solaire pour devenir l'avatar d'une entité céleste aujourd'hui oubliée. Transfiguré, il obtint des ailes dorées comme la lumière de l'aube. Son armure étincelait comme une constellation d'espoir d'au-delà d

In [59]:
df = pd.read_csv("../data/processed/lore_chunked.csv", sep="\t")

In [71]:
vectorstore = FAISS.load_local("../data/vectorstores/faiss_index", embedding_model, allow_dangerous_deserialization=True)
print("Nombre total de vecteurs :", vectorstore.index.ntotal)

Nombre total de vecteurs : 7792


We split the user_query and use rapid fuzz token ratio to filter on the champion

In [72]:
from rapidfuzz import process, fuzz

user_query = "C'est quoi l'histoire de atrox"  
splitted_input = user_query.split()
all_names = df["Name"].unique()

for element in splitted_input:
    best_match_name, score, _ = process.extractOne(element.upper(), all_names, scorer=fuzz.token_ratio)
    if score > 90:
        champion_detected = best_match_name

champion_detected


'AATROX'

In [73]:
chunks_for_champion = df[df["Name"] == champion_detected]

# Créer un vectorstore temporaire pour ce champion
vectorstore_filtered = FAISS.from_documents(
    [Document(page_content=row["page_content"], metadata=row.to_dict())
     for _, row in chunks_for_champion.iterrows()],
    embedding_model
)


In [74]:
results = vectorstore_filtered.similarity_search(user_query, k=10)
print(len(results))
for r in results:
    print(r.page_content[:200])  

10
porteur d'épée. D'un meilleur réceptacle. Autour de moi, il n'y a que des morts et des agonisants. J'entends leurs âmes se retirer de ce monde. Le combat n'a pas pris fin. Il continue dans l'enceinte 
déchus soient aussi dangereux pour la survie de Runeterra que les incursions du Néant, les Targoniens intervinrent. On prétend que la Manifestation du crépuscule donna aux mortels la connaissance néce
Qu'on la prenne pour un dieu ou pour un démon, l'Épée des Darkin a fait l'objet de nombreux contes... mais peu connaissent son nom véritable ou l'histoire de sa chute. Aux temps les plus reculés, avan
mes hôtes. Les ténèbres. Il pleuvait pendant la bataille. Et si la boue me recouvre ? Si je suis enfoui pendant des milliers d'années ? Enfermé dans cette prison ? L'horreur de cette perspective nourr
et la haine grandirent dans son cœur. Les puissances célestes qu'Aatrox avait autrefois incarnées avaient été balayées de ce monde et du souvenir des hommes. Furieux contre cette injustice, Aat

Concatenate all the result to build a context for the RAG

In [75]:
retrieved_chunk = "\n".join([r.page_content for r in results])

In [66]:
import os
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

In [ ]:
prompt = f"""
Le contexte de la question est en dessous.
---------------------
{retrieved_chunk}
---------------------
Tu es un expert geek et connais tout l'histoire du jeu League of Legends. En t'appuyant sur le contexte la question et de la question posée, peux tu répondres de façon claire et concise.
Query: {user_query}
Answer:
"""

"\nLe contexte de la question est en dessous.\n---------------------\nporteur d'épée. D'un meilleur réceptacle. Autour de moi, il n'y a que des morts et des agonisants. J'entends leurs âmes se retirer de ce monde. Le combat n'a pas pris fin. Il continue dans l'enceinte de la ville. J'oblige ma nouvelle forme à ramper vers le plus fort de la bataille. Vers un meilleur\ndéchus soient aussi dangereux pour la survie de Runeterra que les incursions du Néant, les Targoniens intervinrent. On prétend que la Manifestation du crépuscule donna aux mortels la connaissance nécessaire pour piéger les Darkin, et que la Manifestation de la guerre, qui venait de renaître, unifia\nQu'on la prenne pour un dieu ou pour un démon, l'Épée des Darkin a fait l'objet de nombreux contes... mais peu connaissent son nom véritable ou l'histoire de sa chute. Aux temps les plus reculés, avant que les sables du désert n'avalent l'empire, un puissant champion de Shurima fut conduit devant le\nmes hôtes. Les ténèbres. I

In [ ]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)